# Setup

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import json
import numpy as np
from datetime import datetime
from backtesting_utils import (make_results,
                               drawdown, merge,
                               reduce_n_trades_max,
                               disable_code_cells,
                               add_open_trades,
                               get_streaks_runs,
                               map_risk_to_streak,
                               result_by_close_given_risk,
                               challenge_pass)

import cufflinks as cf
cf.set_config_file(offline=True, offline_show_link=False, theme='white')
cf_colors = ['#3F93CA', '#DB3A34', '#466365', '#FFCAB1', '#C0E5C8', '#0b032d']

print(f"Last run: {datetime.now().isoformat()}")

Last run: 2022-03-05T09:47:09.074229


In [2]:
with open('../input/main-test-daniel-dataset-json/test.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

In [3]:
disable_code_cells()

***
# Analisi Risultati

Partiamo ipotizzando di seguire "alla lettera" (ammesso un certo margine di errore) i segnali e iniziamo con lo studiare le due varianti più importanti:
- Alla lettera, chiudendo l'intera posizione a TP1
- Alla lettera, chiudendo il 60% della posizione a TP1 e il 40% a TP2

In [4]:
df_60_40 = make_results(data[:], partials=[0.6, 0.4], only=['close', 'result'])
df_100 = make_results(data[:], partials=[1], only=['close', 'result'])

In [5]:
df_60_40_by_close = df_60_40.set_index('close').sort_index()
df_100_by_close = df_100.set_index('close').sort_index()

merged = merge({'60_40': df_60_40_by_close, '100': df_100_by_close})   
merged.cumsum().iplot(kind='spread', connectgaps=True, interpolation='hv', 
                      title='Risultato cumulativo (R) [by Close]', width=1.5, legend='top',
                      yTitle='R', fill=False, zerolinecolor='grey', colors=cf_colors)

In [6]:
df = merged.copy().groupby(pd.Grouper(freq='M')).sum().reset_index()
df['close'] = df.close.dt.strftime('%b %Y')
df.iplot(kind='bar', x='close', opacity=1, title='Risultato mensile (R) [by Close]',
         yTitle='R', zerolinecolor='grey', colors=cf_colors, xTitle='month', legend='top')

In [7]:
df = merged.copy().groupby(pd.Grouper(freq='D')).sum()
df = df[df!=0].dropna(how='all').reset_index()
n_days = (df.iloc[-1, 0] - df.iloc[0, 0]).days
df['close'] = df.close.dt.strftime('%d %b %Y')
df.iplot(kind='bar', x='close', opacity=1, yTitle='R', zerolinecolor='grey',
         title=f'Risultato giornaliero (R) [by Close] [non_zero={len(df)}/{n_days}] [worked={len(df)/(n_days*.7):.0%}]',
         colors=cf_colors, xTitle='date', legend='top')

In [8]:
merged = merge({'60_40': drawdown(df_60_40_by_close),
               '100': drawdown(df_100_by_close)})

merged.reset_index(drop=True).iplot(kind='scatter', connectgaps=True, interpolation='hv', colors=cf_colors, fill=False,
                                    title='Drawdown (R) [by Close]', xTitle='index', yTitle='R', legend='bottom', width=1.5)

### Per questi prossimi grafici non è necessario analizzare la differenza fra le due versioni

In [9]:
df = df_100.copy().groupby(pd.Grouper(freq='D')).count()
df = df[df!=0].dropna(how='all')
df['result'].iplot(kind='hist', histnorm='probability', colors=cf_colors, opacity=1, xTitle='trades',
                   title='Distribuzione del numero di trades in 1 giorno (frequenza) [by Open]',
                   yTitle='probability', bargroupgap=0.2, linecolor='grey', zerolinecolor='grey')

In [10]:
df['result'] = df_100.copy().groupby(pd.Grouper(freq='D')).sum()['result']
df.groupby('close').sum().loc[1:, 'result'].iplot(kind='bar', opacity=1, colors=cf_colors,
                                                  title='Risultato cumulativo in base al numero di trade giornalieri (R) [by Open]',
                                                  yTitle='R', zerolinecolor='grey')

In [11]:
df = df_100.copy()
df['counter'] = 1
df['counter'] = df[['counter']].groupby(pd.Grouper(freq='D')).transform(lambda x: x.cumsum())
df.groupby('counter').sum().loc[1:, 'result'].iplot(kind='bar', opacity=1, colors=cf_colors,
                                                  title='Risultato cumulativo in base alla posizione del trade nella giornata (R) [by Open]',
                                                  yTitle='R', zerolinecolor='grey')

***
## Limite al numero di trades giornalieri
E' chiaro che il numero di trades giornalieri influisce sul risultato della strategia:
 - Proviamo a limitire il numero massimo di operazioni giornaliere
 - Paragonerò la strategia 60_40 visto che si è rivelata più profittevole

In [12]:
max_1_60_40_by_close = reduce_n_trades_max(df_60_40, 1).set_index('close').sort_index()
max_2_60_40_by_close = reduce_n_trades_max(df_60_40, 2).set_index('close').sort_index()
max_3_60_40_by_close = reduce_n_trades_max(df_60_40, 3).set_index('close').sort_index()
max_4_60_40_by_close = reduce_n_trades_max(df_60_40, 4).set_index('close').sort_index()
merged = merge({'60_40_max_1': max_1_60_40_by_close,
                '60_40_max_2': max_2_60_40_by_close,
                '60_40_max_3': max_3_60_40_by_close,
                '60_40_max_4': max_4_60_40_by_close,
                '60_40_old': df_60_40_by_close})

In [13]:
merged.cumsum().iplot(kind='scatter', connectgaps=True, interpolation='hv', 
                      title='Risultato cumulativo (R) [by Close]', width=1.5, legend='top',
                      yTitle='R', fill=False, zerolinecolor='grey', opacity=1, colors=cf_colors)

*E' chiaro che la combinazione migliore è **60_40** con **max 3 trade al giorno**, quindi proseguiamo ad analizzare quella.*

In [14]:
merged = merge({'max_3': max_3_60_40_by_close,
                'old': df_60_40_by_close})

In [15]:
df = merged.copy().groupby(pd.Grouper(freq='D')).sum()
df = df[df!=0].dropna(how='all').reset_index()
n_days = (df.iloc[-1, 0] - df.iloc[0, 0]).days
df['close'] = df.close.dt.strftime('%d %b %Y')
df.iplot(kind='bar', x='close', opacity=1, yTitle='R', zerolinecolor='grey', subplots=True,
         title=f'Risultato giornaliero (R) [by Close] [non_zero={len(df)}/{n_days}] [worked={len(df)/(n_days*.7):.0%}]',
         colors=cf_colors, xTitle='date', legend='top', shared_yaxis=True)

In [16]:
merged = merge({'max_3': drawdown(max_3_60_40_by_close),
               'old': drawdown(df_60_40_by_close)})

merged.reset_index(drop=True).iplot(kind='scatter', connectgaps=True, interpolation='hv', colors=cf_colors, fill=False,
                                    title='Drawdown (R) [by Close]', xTitle='index', yTitle='R', legend='bottom', width=1.5)

## Iniziamo a ragionare in percentuali
### Ora che il rischio massimo giornaliero è stato limitato, possiamo pensare di aumentare il rischio individuale per trade
- Se il rischio non è specificato intendo 1%
- Includiamo anche il caso di max 2 trades al giorno ma moltiplicatore di rischio più alto

In [17]:
max_3_60_40_risk_1_5_by_close = max_3_60_40_by_close.copy()
max_2_60_40_risk_2_by_close = max_2_60_40_by_close.copy()
max_3_60_40_risk_1_5_by_close['result'] *= 1.5
max_2_60_40_risk_2_by_close['result'] *= 2
merged = merge({'max_3': max_3_60_40_by_close,
                'old': df_60_40_by_close,
                'max_3 r1.5%': max_3_60_40_risk_1_5_by_close,
                'max_2 r2%': max_2_60_40_risk_2_by_close})

In [18]:
merged.cumsum().iplot(kind='scatter', connectgaps=True, interpolation='hv', 
                      title='Risultato cumulativo (%) [by Close]', width=1.5, legend='top',
                      yTitle='%', fill=False, zerolinecolor='grey', colors=cf_colors)

In [19]:
df = merged.copy().groupby(pd.Grouper(freq='M')).sum().reset_index()
df['close'] = df.close.dt.strftime('%b %Y')
df.iplot(kind='bar', x='close', opacity=1, title='Risultato mensile (%) [by Close]',
         yTitle='%', zerolinecolor='grey', colors=cf_colors, xTitle='month', legend='top')

E' chiaro che a Dicembre 2020 non fare il terzo trade giornaliero abbia impattato moltissimo, vediamo la distribuzione dei risultati per posizione del trade all'interno della giornata per il mese di Dicembre 2020:

In [20]:
df = df_100.copy()
df['counter'] = 1
df['counter'] = df[['counter']].groupby(pd.Grouper(freq='D')).transform(lambda x: x.cumsum())
df = df.loc[(df.index > '2020-12-1') & (df.index < '2021-1-1')].groupby('counter').sum().loc[1:, 'result']
df.astype(int).iplot(kind='bar', opacity=1, colors=cf_colors,
                     title='Risultato cumulativo in base alla posizione del trade nella giornata, Dicembre 2020 (R) [by Open]',
                     yTitle='R', zerolinecolor='grey', bargroupgap=0.5)

In [21]:
merged = merge({'max_3': drawdown(max_3_60_40_by_close),
               'old': drawdown(df_60_40_by_close),
               'max_3 r1.5%': drawdown(max_3_60_40_risk_1_5_by_close),
               'max_2 r2%': drawdown(max_2_60_40_risk_2_by_close)})

merged.reset_index(drop=True).iplot(kind='scatter', connectgaps=True, interpolation='hv', colors=cf_colors, fill=False,
                                    title='Drawdown (%) [by Close]', xTitle='index', yTitle='%', legend='bottom', width=1.5)

***
## Risk Management Dinamico

In [22]:
merged = merge({'100_max_3': add_open_trades(reduce_n_trades_max(df_100, 3))[['open_trades']],
                '100_max_2': add_open_trades(reduce_n_trades_max(df_100, 2))[['open_trades']],
                '100': add_open_trades(df_100)[['open_trades']]}).reset_index(drop=True)

non_one = sum([len(merged[(merged[f'{x}'].notna()) & (merged[f'{x}'] != 1)]) / 
               len(merged[merged[f'{x}'].notna()]) for x in merged.columns]) / len(merged.columns)

merged.iplot(kind='bar', mode='group', subplots=True, shared_yaxis=True, legend='top', colors=cf_colors,
             title=f'Posizioni aperte in un dato momento [by Open] [non_one={non_one:.1%}]')

### Streak Analysis

Quando si studiano le streak, è importante tenere a mente che un trade (2) può essere stato aperto prima che il trade precendente (1) venisse chiuso, e quindi in realtà non si saprebbe l'esito del trade (1) nel momento in cui si apre (2). Come si vede dal grafico sopra, nel caso di Daniel questo avviene raramente (6.1% dei casi).

In [23]:
merged = merge({'normal': get_streaks_runs(df_100),
                'max_2': get_streaks_runs(reduce_n_trades_max(df_60_40, 2)),
                'max_3': get_streaks_runs(reduce_n_trades_max(df_60_40, 3))})
merged.iplot(kind='hist', barmode='group', title='Distribuzione streak [by Open]', yTitle='probability',
             histnorm='probability', legend='top', colors=cf_colors, opacity=1, xTitle='streak length')

### Consideriamo questa strategia di rischio (1):
- Se abbiamo perso 1 volta: rischio=0.75%
- Se abbiamo perso 3 volte di fila: rischio=0.75%
- Se abbiamo vinto 1 volta: rischio=2%
- Se abbiamo vinto 3 volta di fila: rischio=2%

In [24]:
risk = {-1: 0.75, -3: 0.75, 1: 2, 3: 2}
map_risk_to_streak(df_100, risk)

,close,result,result_type,risk
open,,,,
2020-11-05 12:46:00+00:00,2020-11-05 13:05:00+00:00,-1.000000,SL,1.00
2020-11-05 13:26:28+00:00,2020-11-05 13:42:00+00:00,-1.003650,SL,0.75
2020-11-06 02:08:20+00:00,2020-11-06 02:31:00+00:00,-0.036885,CLOSE,1.00
2020-11-09 09:32:23+00:00,2020-11-09 09:57:00+00:00,-0.036290,SL_TO_BE,0.75
2020-11-10 23:38:12+00:00,2020-11-11 00:06:00+00:00,-1.000000,SL,1.00
...,...,...,...,...
2022-01-25 08:18:06+00:00,2022-01-25 09:22:00+00:00,0.988938,TP,2.00
2022-01-26 10:14:32+00:00,2022-01-26 11:20:00+00:00,0.877934,TP,1.00
2022-01-27 09:44:42+00:00,2022-01-27 10:37:00+00:00,0.982234,TP,2.00


In [25]:
risk = {-1: 0.75, -3: 0.75, 1: 2, 3: 2}

musketeers = {'normal': result_by_close_given_risk(map_risk_to_streak(df_100, risk)),
              'max_2': result_by_close_given_risk(map_risk_to_streak(reduce_n_trades_max(df_60_40, 2), risk)),
              'max_3': result_by_close_given_risk(map_risk_to_streak(reduce_n_trades_max(df_60_40, 3), risk))}

merged = merge(musketeers)

merged.cumsum().iplot(kind='scatter', connectgaps=True, interpolation='hv', colors=cf_colors, fill=False,
                      title='Return (%) [by Close]', xTitle='index', yTitle='%', legend='top', width=1.5, zerolinecolor='grey')

In [26]:
df = merged.copy().groupby(pd.Grouper(freq='M')).sum().reset_index()
df['close'] = df.close.dt.strftime('%b %Y')
df.iplot(kind='bar', x='close', opacity=1, title='Risultato mensile (%) [by Close]',
         yTitle='%', zerolinecolor='grey', colors=cf_colors, xTitle='month', legend='top')

In [27]:
merged = merge({'normal': drawdown(musketeers['normal']),
                'max_2': drawdown(musketeers['max_2']),
                'max_3': drawdown(musketeers['max_3'])})

merged.reset_index(drop=True).iplot(kind='scatter', connectgaps=True, interpolation='hv', colors=cf_colors, fill=False,
                      title='Drawdown (%) [by Close]', xTitle='index', yTitle='%', legend='bottom', width=1.5)

### Breve assessment del progresso fatto fino ad ora:
L'istogramma qui sotto rappresenta la distribuzione dei risultati delle Challenge se ne avessimo fatta una per ogni giorno lavorativo del dataset (esclusi i primi e ultimi 30 giorni).

In [28]:
merged = merge({'vanilla_100': challenge_pass(df_100),
                'vanilla_60_40': challenge_pass(df_60_40),
                'std_risk_60_40_max_2': challenge_pass(reduce_n_trades_max(df_60_40, 2)),
                'std_risk_60_40_max_3': challenge_pass(reduce_n_trades_max(df_60_40, 3)),
                'dyn_risk_60_40_max_2': challenge_pass(result_by_close_given_risk(map_risk_to_streak(reduce_n_trades_max(df_60_40, 2), risk))),
                'dyn_risk_60_40_max_3': challenge_pass(result_by_close_given_risk(map_risk_to_streak(reduce_n_trades_max(df_60_40, 3), risk)))})

merged.iplot(kind='hist', barmode='group', colors=cf_colors, legend='top', histnorm='probability', xTitle='pass', yTitle='probability',
             title='Challenge pass rate FTMO (1=win, 0=not failed, -1=failed) [by Close]', opacity=1)

In [29]:
merged = merge({'vanilla_100': challenge_pass(df_100, max_dd=-.12, target=.08),
                'vanilla_60_40': challenge_pass(df_60_40, max_dd=-.12, target=.08),
                'std_risk_60_40_max_2': challenge_pass(reduce_n_trades_max(df_60_40, 2), max_dd=-.12, target=.08),
                'std_risk_60_40_max_3': challenge_pass(reduce_n_trades_max(df_60_40, 3), max_dd=-.12, target=.08),
                'dyn_risk_60_40_max_2': challenge_pass(result_by_close_given_risk(map_risk_to_streak(reduce_n_trades_max(df_60_40, 2), risk)), max_dd=-.12, target=.08),
                'dyn_risk_60_40_max_3': challenge_pass(result_by_close_given_risk(map_risk_to_streak(reduce_n_trades_max(df_60_40, 3), risk)), max_dd=-.12, target=.08)})

merged.iplot(kind='hist', barmode='group', colors=cf_colors, legend='top', histnorm='probability', xTitle='pass', yTitle='probability',
             title='Challenge pass rate MFF (1=win, 0=not failed, -1=failed) [by Close]', opacity=1)